In [2]:
import pandas as pd
import plotting as plotting
from plotting import ConvertAndWrite
from plotting.plotMaster import plotMaster
import numpy as npz
from scipy.optimize import curve_fit
%matplotlib inline

ImportError: cannot import name 'ConvertAndWrite'

In [ ]:
filename = '~/Desktop/water_phantom/LT_4802297_021920'
#filename = 'LU_4802295_021920'
odir = 'output'
df = pd.read_csv('{}.csv'.format(filename))
df_out = df.copy()

In [ ]:
settings = {
    'plotBoxOff': 1,
    'useLogXAxis': 0,
    'useLogYAxis': 0,
    'doSvg': 1,
    'textLocation': [0.008, 0.97],
    'xlabel': 'Echo Time',
    'ylim': [None, None],
    'title': '',
    'ylabel': 'Signal',
    #'xticks': np.array([  4.,  24.,  48.,  96., 168.]),
    #'xticklabels': ['4', '24', '48', '96', '168'],
    'rotate': 0,
    'alpha': [1, 1,1,1],
    'linestyle': ['','','-','-'],
    'marker': ['o','o','',''],
    'text': '',
    'basename': 'tmp'

}
legend = {'names': ['In Phase','Out Phase','In Fit','Out Fit'],
         'location': 1,
         'bb': (1, 1),
         'columns': 1,
         'fontsize': 16}

In [ ]:
def oneExpC(t,a,b,c):
    return a*np.exp(-(1/b)*t)+c
def oneExp(t,a,b):
    return a*np.exp(-(1/b)*t)

In [ ]:
in_phase = np.array([(x[1]['Echo Time'],x[1]['Water Mean']) for x in df[df['Type'] == 'inphase'].iterrows()],dtype=[('Time','f4'),('Value','f4')])
out_phase = np.array([(x[1]['Echo Time'],x[1]['Water Mean']) for x in df[df['Type'] == 'outphase'].iterrows()],dtype=[('Time','f4'),('Value','f4')])

In [ ]:
in_fit = curve_fit(oneExpC,in_phase["Time"],in_phase['Value'],bounds=([0,0,0],[np.inf,np.inf,np.inf]))
out_fit = curve_fit(oneExpC,out_phase["Time"],out_phase['Value'],bounds=([0,0,0],[np.inf,np.inf,np.inf]))
in_fit = curve_fit(oneExp,in_phase["Time"],in_phase['Value'],bounds=([0,0],[np.inf,np.inf]))
out_fit = curve_fit(oneExp,out_phase["Time"],out_phase['Value'],bounds=([0,0],[np.inf,np.inf]))

In [ ]:
in_phaseI = np.linspace(in_phase['Time'][0],in_phase['Time'][-1])
out_phaseI = np.linspace(out_phase['Time'][0],out_phase['Time'][-1])
fin_phase = oneExp(in_phaseI,in_fit[0][0],in_fit[0][1])
fout_phase = oneExp(out_phaseI,out_fit[0][0],out_fit[0][1])

In [ ]:
fac = in_fit[0][0]/out_fit[0][0]
fac = np.mean(oneExp(out_phase['Time'],*in_fit[0])/out_phase['Value'])
idx = df_out['Type'] == 'outphase'
for organ in ['Kidneys','Liver','Spleen','Water']:
    df_out.loc[idx,'{} Mean'.format(organ)] = df_out.loc[idx,'{} Mean'.format(organ)].map(lambda x: x * fac)
df_out.to_csv('{}/{}-corected.csv'.format(odir,filename))

In [ ]:
settings['basename'] = '{}/{}-raw'.format(odir,filename)
fig,ax,leg = plotMaster([in_phase['Time'],out_phase['Time'],in_phaseI,out_phaseI],
                         [in_phase['Value'],out_phase['Value'],fin_phase,fout_phase],
                       [],[],
                       plot_settings=settings,
                       plot_type='plot',
                       legend=legend)
ConvertAndWrite(settings,fig)

In [ ]:
settings['basename'] = '{}/{}-corr'.format(odir,filename)
fig,ax,leg = plotMaster([in_phase['Time'],out_phase['Time'],in_phaseI,out_phaseI],
                         [in_phase['Value'],out_phase['Value']*fac,fin_phase,fout_phase*fac],
                       [],[],
                       plot_settings=settings,
                       plot_type='plot',
                       legend=legend)
ConvertAndWrite(settings,fig)

In [ ]:
fac

In [ ]:
df

In [ ]:
df_out

In [ ]:
758*1.06